# Title
[]()

In [1]:
import requests
import json
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
import pickle
import os


In [2]:
from datetime import datetime
from plotly.subplots import make_subplots

## Load processed data as dataframe

In [3]:
json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw'
response_json2022 = loadpickle('my_ig_posts_2022_pulled_2023-02-11.sav', json_path)
len(response_json2022)

Time completed: 2023-02-22 23:44:52.723483


2

In [4]:
from processing import *
def response_json_dict_to_df(response_json_dict):
    """
    Convert the data from the response_json_dict (e.g. from get_user_ig_post_text function) 
    into a DataFrame.
    """
    data = pd.concat([json_normalize(response['data']) for key, response in response_json_dict.items()])
    data_processed, count_vector, vect = post_preprocessing(data)
    return data_processed, count_vector, vect

df2022_processed, count_vector, vect = response_json_dict_to_df(response_json2022)
df2022_processed.head()

Token pattern: (?u)\b\w\w+\b
Shape of vector array:  (41, 660)

Top 10 words:
<number>    30
time        14
good        11
exercise    11
train        9
get          9
work         9
muscle       9
people       9
day          8
dtype: int64
Time processed: 2023-02-22 23:45:58.439000


,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url,date,year,month,week_of_year,day_of_week,time,hour
0,2022-12-30 02:05:19+00:00,💛,30,0,CAROUSEL_ALBUM,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CmxlFotJyli/,18336168253034275,NaN,NaN,2022-12-30,2022,12,52,4,02:05:19,2
1,2022-12-27 20:48:37+00:00,power come responsibility fitness space peop...,5,2,IMAGE,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/Cmr3QXSSDdE/,18033248329425680,"[{'timestamp': '2022-12-27T21:04:59+0000', 'te...",NaN,2022-12-27,2022,12,52,1,20:48:37,20
2,2022-12-27 03:31:51+00:00,podcast fee mostly fill podcast data science ...,5,1,IMAGE,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CmqAm0tL1u1/,17980063225869664,"[{'timestamp': '2022-12-28T16:45:59+0000', 'te...",NaN,2022-12-27,2022,12,52,1,03:31:51,3
3,2022-12-22 19:35:45+00:00,here link super data science podcast discus es...,4,0,CAROUSEL_ALBUM,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/Cme28PFuJ57/,17926236674624239,NaN,NaN,2022-12-22,2022,12,51,3,19:35:45,19
4,2022-12-21 22:44:24+00:00,neighbor house long sharp icicle hang along ro...,11,5,IMAGE,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CmcnvFpOYeN/,17939812733373203,"[{'timestamp': '2022-12-24T20:57:54+0000', 'te...",NaN,2022-12-21,2022,12,51,2,22:44:24,22


In [73]:
df2022_processed.dtypes

timestamp             datetime64[ns, UTC]
caption                            object
like_count                          int64
comments_count                      int64
media_type                         object
media_product_type                 object
media_url                          object
permalink                          object
id                                 object
comments.data                      object
thumbnail_url                      object
date                               object
year                                int64
month                               int64
week_of_year                       UInt32
day_of_week                         int64
time                               object
hour                                int64
dtype: object

# `plot_images`

## version 1

In [63]:
def plot_images(df, n=6, top=True, max_columns=5, streamlit=False):
    """
    Plot the images/video thumbnails of either the top or 
    worst performing instagram media (posts, reels, carousels).
    """
    ncols = n if n<max_columns else max_columns
    nrows = (n + ncols - 1) // ncols
    sort_by = ['like_count', 'comments_count', 'timestamp']
    posts = df.sort_values(by=sort_by, ascending=False if top else True).head(n).copy()
    posts['thumbnail_url'].fillna(posts['media_url'], inplace=True)
    titles = tuple(posts['timestamp'].dt.strftime('%Y-%m-%d at %H:%M').values.tolist())
    fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=titles)
    for index, (n_likes, n_comments, url) in enumerate(zip(
            posts['like_count'], posts['comments_count'], posts['thumbnail_url'])
        ):
        fig.add_layout_image(
            x=0, y=0,
            xanchor='center', yanchor='middle',
            sizex=1, sizey=1,
            row=index // ncols + 1,
            col=index % ncols + 1,
            xref="x",
            yref="y",
            opacity=1.0,
            source=url
        )
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=-.10,
            text=f'{n_likes} likes, {n_comments} comments',
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
    fig.update_xaxes(range=[-0.5,0.5], showticklabels=False)
    fig.update_yaxes(range=[-0.5,0.5], showticklabels=False)
    fig.update_layout(plot_bgcolor="white")
    if streamlit:
        st.plotly_chart(fig, use_container_width=True)
    else:
        fig.show()
    return posts.reset_index(drop=True)

posts = plot_images(df2022_processed, n=3, max_columns=5)

In [66]:
posts.head()

,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,thumbnail_url,date,year,month,week_of_year,day_of_week,time,hour
0,2022-09-16 19:44:31+00:00,yell improve strength thank <Handle> coach t...,201,8,VIDEO,REELS,https://video.cdninstagram.com/v/t50.33967-16/...,https://www.instagram.com/reel/CilGehzAgG3/,17959530130960346,"[{'timestamp': '2022-10-25T19:20:29+0000', 'te...",https://scontent.cdninstagram.com/v/t51.29350-...,2022-09-16,2022,9,37,4,19:44:31,19
1,2022-01-29 23:37:52+00:00,couple time someone ￼recognized mask impress ...,64,4,IMAGE,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CZVStgnvrAn/,18148518601242451,"[{'timestamp': '2022-01-30T01:26:16+0000', 'te...",https://scontent.cdninstagram.com/v/t51.29350-...,2022-01-29,2022,1,4,5,23:37:52,23
2,2022-09-13 20:50:18+00:00,congrats suzanne eric find happiness get marry...,36,4,CAROUSEL_ALBUM,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/Cidf-z3vR6G/,17923493357545447,"[{'timestamp': '2022-09-14T06:24:49+0000', 'te...",https://scontent.cdninstagram.com/v/t51.29350-...,2022-09-13,2022,9,37,1,20:50:18,20


In [74]:
posts.dtypes

timestamp             datetime64[ns, UTC]
caption                            object
like_count                          int64
comments_count                      int64
media_type                         object
media_product_type                 object
media_url                          object
permalink                          object
id                                 object
comments.data                      object
thumbnail_url                      object
date                               object
year                                int64
month                               int64
week_of_year                       UInt32
day_of_week                         int64
time                               object
hour                                int64
dtype: object

In [75]:
[timestamp for timestamp in pd.to_datetime(posts['timestamp'])]

[Timestamp('2022-09-16 19:44:31+0000', tz='UTC'),
 Timestamp('2022-01-29 23:37:52+0000', tz='UTC'),
 Timestamp('2022-09-13 20:50:18+0000', tz='UTC')]

In [76]:
[timestamp.astimezone('Australia/Sydney') for timestamp in pd.to_datetime(posts['timestamp'])]

[Timestamp('2022-09-17 05:44:31+1000', tz='Australia/Sydney'),
 Timestamp('2022-01-30 10:37:52+1100', tz='Australia/Sydney'),
 Timestamp('2022-09-14 06:50:18+1000', tz='Australia/Sydney')]

In [77]:
[timestamp.astimezone('Canada/Pacific') for timestamp in pd.to_datetime(posts['timestamp'])]

[Timestamp('2022-09-16 12:44:31-0700', tz='Canada/Pacific'),
 Timestamp('2022-01-29 15:37:52-0800', tz='Canada/Pacific'),
 Timestamp('2022-09-13 13:50:18-0700', tz='Canada/Pacific')]

In [79]:
pd.to_datetime(posts['timestamp']).values.astimezone('Canada/Pacific')

AttributeError: 'numpy.ndarray' object has no attribute 'astimezone'

In [65]:
import pytz
  
print('Commonly used time-zones:', pytz.common_timezones, '\n')

Commonly used time-zones: ['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'America/Adak', 

## Version 2

In [87]:
def plot_images(df, n=6, top=True, max_columns=5, streamlit=False, timezone='Canada/Pacific'):
    """
    Plot the images/video thumbnails of either the top or 
    worst performing instagram media (posts, reels, carousels).
    The following data is shown for each resulting media item:
        - Image/video thumbnail
        - Time stamp of the post in the provided time zone or in UTC time.
        - Number of likes and number of comments.

    Parameters:
        - df: DataFrame with the processed data.
        - n (int): Number of images to show.
        - top (bool): If True, plot images with the highest number of likes in 
            descending order. If False, plot images with the highest number of likes in
            ascending order. sort_by = ['like_count', 'comments_count', 'timestamp']
        - streamlit (bool): Whether or not the app runs on Streamlit. If False, then
            it is run on local machine.
        - timezone (str): Timezone parameter for the `.astimezone()` method.
            e.g. 'Australia/Sydney', 'Canada/Pacific'
    Returns:
        - DataFrame containing the data of the posts in the figure.
        - fig: Plotly figure object.
    """
    ncols = n if n<max_columns else max_columns
    nrows = (n + ncols - 1) // ncols
    sort_by = ['like_count', 'comments_count', 'timestamp']
    posts = df.sort_values(by=sort_by, ascending=False if top else True).head(n).copy()
    posts['thumbnail_url'].fillna(posts['media_url'], inplace=True)
    if timezone:
        print('Time zone:', timezone)
        converted_timestamp = [timestamp.astimezone(timezone) for timestamp in pd.to_datetime(posts['timestamp'])]
        titles = tuple(timestamp.strftime('%Y-%m-%d at %H:%M') for timestamp in converted_timestamp)
    else:
        titles = tuple(posts['timestamp'].dt.strftime('%Y-%m-%d at %H:%M').values.tolist())
    fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=titles)
    for index, (n_likes, n_comments, url) in enumerate(zip(
            posts['like_count'], posts['comments_count'], posts['thumbnail_url'])
        ):
        fig.add_layout_image(
            x=0, y=0,
            xanchor='center', yanchor='middle',
            sizex=1, sizey=1,
            row=index // ncols + 1,
            col=index % ncols + 1,
            xref="x",
            yref="y",
            opacity=1.0,
            source=url
        )
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=-.10,
            text=f'{n_likes} likes, {n_comments} comments',
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
    fig.update_xaxes(range=[-0.5,0.5], showticklabels=False)
    fig.update_yaxes(range=[-0.5,0.5], showticklabels=False)
    fig.update_layout(plot_bgcolor="white")
    if streamlit:
        st.plotly_chart(fig, use_container_width=True)
    else:
        fig.show()
    return posts.reset_index(drop=True), fig

posts, fig = plot_images(df2022_processed, n=3, max_columns=5, timezone=None)

# Plot with top Tfidf scores

## tfidf_transform

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
def tfidf_transform(count_vector):
    vectorizer = TfidfTransformer()
    tfidf= pd.DataFrame(
        vectorizer.fit_transform(count_vector).toarray(), 
        columns=vectorizer.get_feature_names_out()
        )
    return tfidf

tfidf = tfidf_transform(count_vector)
tfidf['ab'].sort_values(ascending=False).head(3)

40    0.065557
21    0.000000
23    0.000000
Name: ab, dtype: float64

In [16]:
tfidf.loc[1].sort_values(ascending=False).index

Index(['youll', 'excite', 'ai', 'health', 'recommend', 'get', 'adopt',
       'challenge', 'people', 'app',
       ...
       'fountain', 'frailty', 'free', 'friend', 'front', 'functional', 'glue',
       'glut', 'go', '<number>'],
      dtype='object', length=660)

## plot

In [10]:
def plot_images_tfidf(df, count_vector,
        n=6, top=True, max_columns=5, streamlit=False, timezone='Canada/Pacific'):
    """
    Plot the images/video thumbnails of either the top or 
    worst performing instagram media (posts, reels, carousels).
    The following data is shown for each resulting media item:
        - Image/video thumbnail
        - Time stamp of the post in the provided time zone or in UTC time.
        - Number of likes and number of comments.
        - 5 words with the highest Tf-idf scores.

    Parameters:
        - df: DataFrame with the processed data.
        - count_vector: DataFrame with the count vectors.
        - n (int): Number of images to show.
        - top (bool): If True, plot images with the highest number of likes in 
            descending order. If False, plot images with the highest number of likes in
            ascending order. sort_by = ['like_count', 'comments_count', 'timestamp']
        - streamlit (bool): Whether or not the app runs on Streamlit. If False, then
            it is run on local machine.
        - timezone (str): Timezone parameter for the `.astimezone()` method.
            e.g. 'Australia/Sydney', 'Canada/Pacific'
    Returns:
        - DataFrame containing the data of the posts in the figure.
        - fig: Plotly figure object.
    """
    tfidf = tfidf_transform(count_vector)
    ncols = n if n<max_columns else max_columns
    nrows = (n + ncols - 1) // ncols
    sort_by = ['like_count', 'comments_count', 'timestamp']
    df = pd.concat([df, tfidf], axis=1)
    posts = df.sort_values(by=sort_by, ascending=False if top else True).head(n).copy()
    posts['thumbnail_url'].fillna(posts['media_url'], inplace=True)
    if timezone:
        print('Time zone:', timezone)
        converted_timestamp = [timestamp.astimezone(timezone) for timestamp in pd.to_datetime(posts['timestamp'])]
        titles = tuple(timestamp.strftime('%Y-%m-%d at %H:%M') for timestamp in converted_timestamp)
    else:
        titles = tuple(posts['timestamp'].dt.strftime('%Y-%m-%d at %H:%M').values.tolist())
    fig = make_subplots(
        rows=nrows, cols=ncols, subplot_titles=titles, 
        vertical_spacing=0.55/nrows
        )
    for index, (i, n_likes, n_comments, url, caption) in enumerate(zip(posts.index,
            posts['like_count'], posts['comments_count'], posts['thumbnail_url'],
            posts['caption'])
        ):
        highest_tfidf = [word for word, value in tfidf.loc[i].sort_values(
            ascending=False).head(5).items() if value > 0]
        fig.add_layout_image(
            x=0, y=0,
            xanchor='center', yanchor='middle',
            sizex=1, sizey=1,
            row=index // ncols + 1,
            col=index % ncols + 1,
            xref="x",
            yref="y",
            opacity=1.0,
            source=url
        )
        annotation_y_position = -0.075
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=annotation_y_position,
            text=f'{n_likes} likes, {n_comments} comments',
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
        annotation_y_position -= 0.1
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=annotation_y_position,
            text=f'Most unique words:',
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
        for word in highest_tfidf:
            annotation_y_position -= .075
            fig.add_annotation(
                xref="x domain",
                yref="y domain",
                x=0.5,
                y=annotation_y_position,
                text=word,
                axref="x domain",
                ayref="y",
                ax=0.5,
                ay=2,
                arrowhead=2,
                row=index // ncols + 1,
                col=index % ncols + 1,
                # hovertext=caption,
            )
    fig.update_xaxes(range=[-0.5,0.5], showticklabels=False)
    fig.update_yaxes(range=[-0.5,0.5], showticklabels=False)
    fig.update_layout(
        title_text='Posts with highest number of likes' if top==True else 'Posts with fewest number of likes',
        title_xanchor='center', title_x=0.5,
        plot_bgcolor="white", 
        height = 110 + nrows * 300,
        margin_b=110,
        )
    if streamlit:
        st.plotly_chart(fig, use_container_width=True)
    else:
        fig.show()
    return posts.reset_index(drop=True), fig

posts, fig = plot_images_tfidf(df2022_processed, count_vector, n=6, max_columns=3, timezone=None)

In [9]:
posts.head()

,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,...,#research,#science,#scientist,#stem,#technology,#vancouver,#voluptacommunity,#workout,#yvr,<number>
0,2022-09-16 19:44:31+00:00,yell improve strength thank <Handle> coach t...,201,8,VIDEO,REELS,https://video.cdninstagram.com/v/t50.33967-16/...,https://www.instagram.com/reel/CilGehzAgG3/,17959530130960346,"[{'timestamp': '2022-10-25T19:20:29+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132852
1,2022-01-29 23:37:52+00:00,couple time someone ￼recognized mask impress ...,64,4,IMAGE,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CZVStgnvrAn/,18148518601242451,"[{'timestamp': '2022-01-30T01:26:16+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2022-09-13 20:50:18+00:00,congrats suzanne eric find happiness get marry...,36,4,CAROUSEL_ALBUM,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/Cidf-z3vR6G/,17923493357545447,"[{'timestamp': '2022-09-14T06:24:49+0000', 'te...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [6]:
posts.loc[0]

timestamp                                    2022-09-16 19:44:31+00:00
caption              yell improve strength  thank <Handle>  coach t...
like_count                                                       201.0
comments_count                                                     8.0
media_type                                                       VIDEO
                                           ...                        
#vancouver                                                         NaN
#voluptacommunity                                                  NaN
#workout                                                           NaN
#yvr                                                               NaN
<number>                                                           NaN
Name: 0, Length: 674, dtype: object

# *End of Page*